In [11]:
def outer():
    # ------CLOSURE START--------
    x = 'python'

    def inner():
        print("{0} rocks!".format(x))
    # ------CLOSURE END--------
    return inner # When we return inner, we are actually "returning" the closure

In [12]:
fn = outer()

In [20]:
fn() # When we called fn, at this time Python determined the value of x in the extended scope,
     # But notice that outer had finish running before we called fn - it's scope was "gone"
     # So when outer finish running, the closure still has the value of what x was.

python rocks!


### Python Cells and Multi-Scoped Variables

In [22]:
def outer():
    """
    Here the value of x is shared between two scopes:
        * outer
        * closure
    The label x is in two different scopes but always reference the same "value"

    Python does this by creating a cell as an intermediary object.
    outer.x and inner.x -> cell(0xA500) that is referencing to "0xFF100" -> str(0xFF100) "python"
    """
    x = 'python'   
    def inner():
        print(x)
    return inner

In [28]:
print(outer.__doc__)


    Here the value of x is shared between two scopes:
        * outer
        * closure
    The label x is in two different scopes but always reference the same "value"

    Python does this by creating a cell as an intermediary object.
    outer.x and inner.x -> cell(0xA500) that is referencing to "0xFF100" -> str(0xFF100) "python"
    


### Closures

One can think of the closure as a function plus an extended scope that contains the free variables
The free variable's value is the object the cell points to - so that could change over time.

Everytime the function in the closure is called and the free variable is referenced:
    Python looks up the cell object, and then whatever the cell is pointing to
    But it won't do this until the function is called

In [46]:
def outer():
    a = 100

    x = 'python'
    print(hex(id(x))) # Returns the indirect reference
    def inner():
        a = 10
        print(hex(id(x))) # Returns the indirect reference
        print("{0} rocks!".format(x))
    return inner

In [52]:
fn = outer()

0x107c93780


In [53]:
fn.__code__.co_freevars

('x',)

In [54]:
outer.__code__.co_freevars

()

In [55]:
fn.__closure__

(<cell at 0x10b8dd6c0: str object at 0x107c93780>,)

In [56]:
fn()

0x107c93780
python rocks!


In [58]:
def counter():
    count = 0
    def inc():
        nonlocal count
        count += 1
        return count
    return inc

In [59]:
fn = counter()

In [60]:
fn()

1

In [61]:
fn()

2

### Multiple Instances of Closures

Everytime we run a function, a new scope is created.

If that function generates a closure, a new closure is created everytime as well.

In [62]:
f1 = counter()
f2 = counter()

In [63]:
f1.__closure__

(<cell at 0x10b42c7f0: int object at 0x108b99f88>,)

In [64]:
f2.__closure__

(<cell at 0x10b42cf40: int object at 0x108b99f88>,)

In [65]:
f1()

1

In [66]:
f2()

1

In [67]:
f1.__closure__

(<cell at 0x10b42c7f0: int object at 0x108b99fa8>,)

In [68]:
f2.__closure__

(<cell at 0x10b42cf40: int object at 0x108b99fa8>,)

In [69]:
f2()

2

In [70]:
f2.__closure__

(<cell at 0x10b42cf40: int object at 0x108b99fc8>,)

In [71]:
f1.__closure__

(<cell at 0x10b42c7f0: int object at 0x108b99fa8>,)

### Shared Extended Scopes

In [84]:
def outer():
    count = 0 
    def inc1():
        nonlocal count # count is a free variable - bound to count in the extended scope
        count +=1
        return count
    def inc2():
        nonlocal count # count is a free variable - bound to count in the extended scope
        count += 1
        return count
    return inc1, inc2

In [85]:
f1, f2 = outer()

In [86]:
f1.__closure__

(<cell at 0x10b8dc820: int object at 0x108b99f88>,)

In [87]:
f2.__closure__

(<cell at 0x10b8dc820: int object at 0x108b99f88>,)

In [88]:
f1()

1

In [89]:
f2()

2

In [90]:
f1()

3

In [91]:
f1.__closure__ == f2.__closure__

True

In [92]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [93]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)

In [94]:
add_1(10)

11

In [95]:
add_2(10)

12

In [96]:
add_3(10)

13

In [97]:
add_1.__closure__

(<cell at 0x10b426260: int object at 0x108b99fa8>,)

In [99]:
add_2.__closure__

(<cell at 0x10b425fc0: int object at 0x108b99fc8>,)

In [100]:
add_3.__closure__

(<cell at 0x10b427b20: int object at 0x108b99fe8>,)

In [101]:
adders = []
for n in range(1,4):
    adders.append(lambda x: x + n)

In [102]:
adders

[<function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>]

In [103]:
n

3

In [104]:
adders[0].__closure__

In [105]:
adders[0](10)

13

In [107]:
def create_adders():
    adders = []
    for n in range(1,4):
        adders.append(lambda x: x + n) # this will be evaluated when the lambda runs
    return adders

In [108]:
adders = create_adders()

In [116]:
adders[0].__closure__

(<cell at 0x10b4253c0: int object at 0x108b99fe8>,)

In [117]:
adders[1].__closure__

(<cell at 0x10b4253c0: int object at 0x108b99fe8>,)

In [118]:
adders[2](100)

103

In [119]:
adders[2].__closure__

(<cell at 0x10b4253c0: int object at 0x108b99fe8>,)

In [121]:
adders[0](10)

13

In [124]:
adders[0](42)

45

In [135]:
def create_adders():
    adders = []
    for n in range(1,4):
        adders.append(lambda x, y=n: x + y) # The default values are evaluated at creation time, so at first, y=1 / This way we are not creating closures
    return adders

In [126]:
adders = create_adders()

In [136]:
adders[0](10)

11

In [137]:
adders[0].__closure__

In [138]:
adders[0].__code__.co_freevars

()